In [221]:
from typing import List, Tuple, Union
from datetime import datetime

## Problema 5

Crie um objeto chamado Trade (classe / struct) com os seguintes campos:
TradeDate (formato data), Instrument (formato string), SideIsBuy (booleano
– verdadeiro se foi uma compra), Quantity e Price (ambos formatos numérico,
não necessariamente inteiro).

**Solução:** A primeira parte é bem direta, criar uma classe que represente o objeto Trade. Em Python não existe a noção de atributo/método privado ou público mas, para manter uma boa prática de POO, usei o padrão Python para representar, simbolicamente, um atributo *privado* (__private_attribute). E também, para não acessar os atributos diretamente, implementei *getters* que retornam os valores desses atributos. 

In [287]:
class Trade:
    def __init__(self, TradeDate:datetime, Instrument:str, SideIsBuy:bool, Quantity:Union[float, int], Price:float) -> None:
        
        if not isinstance(TradeDate, datetime): raise TypeError("TradeDate must be of type date.")
        if not isinstance(Instrument, str): raise TypeError("Instrument must be a string.")
        if not isinstance(SideIsBuy, bool): raise TypeError("SideIsBuy must be a boolean.")
        if not (isinstance(Quantity, int) or isinstance(Quantity, float)): raise TypeError("Quantity must be a number (int or float).")
        if not (isinstance(Price, int) or isinstance(Price, float)): raise TypeError("Price must be a number (int or float).")
        
        self.__trade_date = TradeDate
        self.__instrument = Instrument
        self.__side_is_buy = SideIsBuy
        self.__quantity = Quantity
        self.__price = Price

    def get_trade_date(self) -> datetime:
        return self.__trade_date
        
    def get_instrument(self) -> str:
        return self.__instrument
        
    def get_side_is_buy(self) -> bool:
        return self.__side_is_buy
        
    def get_quantity(self) -> float:
        return self.__quantity
        
    def get_price(self) -> float:
        return self.__price


Uma vez definida a classe podemos resolver a parte (a) do problema:

(a) Construa uma Função que recebe uma lista de objetos Trade e retorne o financeiro total (quantidade vezes preço) agrupado por data (venda soma, pois você recebe dinheiro, e compra subtrai). Deve ser uma lista de pares (Data, Financeiro Total) incluindo todas as datas em que ocorreu pelo menos um trade (o financeiro pode ser zero, por exemplo, se houve uma compra e uma venda no mesmo preço).

In [288]:
def total_by_date(trades: Trade) -> List[Tuple[datetime, float]]:

    if not trades: raise ValueError("Enter a Trade list.")
    
    total_for_date = {}

    for trade in trades:

        if trade.get_trade_date().date() > datetime.today().date():
            raise ValueError("Enter a valid Trade! The Trade has not yet occurred and has a future date.")
        
        key = trade.get_trade_date().date()
        total = trade.get_quantity() * trade.get_price() * (1 if trade.get_side_is_buy() else -1)

        if key in total_for_date: total_for_date[key] += total
        else: total_for_date[key] = total

    result = sorted(total_for_date.items())
    return result

assert total_by_date([
    Trade(datetime(2024, 4, 1, 10, 30), "AAPL", True, 10, 150),
    Trade(datetime(2024, 4, 1, 11, 15), "AAPL", False, 5, 160),
    Trade(datetime(2024, 4, 2, 9, 45), "GOOG", True, 8, 250),
    Trade(datetime(2024, 4, 2, 14, 20), "GOOG", False, 3, 260),
    Trade(datetime(2024, 4, 3, 13, 0), "AAPL", True, 15, 155)
]) == [(datetime(2024, 4, 1).date(), 700), (datetime(2024, 4, 2).date(), 1220), (datetime(2024, 4, 3).date(), 2325)], 'The function is not returning the right values.'

assert total_by_date([
    Trade(datetime(2024, 2, 28, 10, 30), "CEAB", True, 10, 100),
    Trade(datetime(2024, 3, 29, 11, 15), "WEG3", True, 5, 160),
    Trade(datetime(2024, 2, 28, 9, 45), "CEAB", False, 8, 50),
    Trade(datetime(2024, 3, 29, 14, 20), "PETR4", False, 3, 260),
    Trade(datetime(2024, 3, 29, 13, 0), "ITUB", True, 15, 155)
]) == [(datetime(2024, 2, 28).date(), 600), (datetime(2024, 3, 29).date(), 2345)], 'The function is not returning the right values.'

assert total_by_date([
    Trade(datetime(2023, 6, 2, 10, 30), "WEG3", True, 10, 150),
    Trade(datetime(2024, 1, 4, 11, 15), "ITSA", False, 5, 160),
    Trade(datetime(2024, 2, 24, 9, 45), "WEG3", True, 8, 250),
    Trade(datetime(2023, 7, 3, 14, 20), "GOOG", False, 3, 260),
    Trade(datetime(2024, 1, 4, 13, 0), "ITUB", True, 15, 155)
]) == [(datetime(2023, 6, 2).date(), 1500), (datetime(2023, 7, 3).date(), -780), (datetime(2024, 1, 4).date(), 1525), (datetime(2024, 2, 24).date(), 2000)], 'The function is not returning the right values.'



(b) Função que recebe uma lista de objetos Trade e uma data (dois parâmetros), e retorne uma lista de pares (Instrument, Quantidade Total) com a posição de cada instrumento na data.

Cada instrumento (string) deve aparecer uma única vez, com a quantidade que  é a soma da quantidade de todas as compras menos a soma das quantidades todas as vendas que ocorreram até a data fornecida (incluindo a própria data). Caso o resultado seja zero, o instrumento deve ser omitido (não deve aparecer na lista).

**Solução:** A lógica aqui é parecida com a anterior, com algumas pequenas mudanças. A principal mudança é adicionar uma verificação de data e mudar nossa *key*.

In [289]:
def total_by_instrument(trades: List[Trade], date: datetime) -> List[Tuple[str, Union[int, float]]]:
    
    if not trades: raise ValueError("Enter a Trade list.")
    
    total_by_instrument = {}

    for trade in trades:

        if trade.get_trade_date().date() > datetime.today().date():
            raise ValueError("Enter a valid Trade! The Trade has not yet occurred and has a future date.")
        
        if trade.get_trade_date().date() == date.date():
            key = trade.get_instrument()
            total = trade.get_quantity() * (1 if trade.get_side_is_buy() else -1)
            
            if key in total_by_instrument: total_by_instrument[key] += total
            else: total_by_instrument[key] = total
                
    return sorted(total_by_instrument.items())

assert total_by_instrument([
    Trade(datetime(2024, 4, 5), "CEAB", True, 100, 150.0),
    Trade(datetime(2024, 4, 5), "CEAB", False, 50, 160.0),
    Trade(datetime(2024, 4, 5), "WEG3", True, 200, 2500.0),
    Trade(datetime(2024, 4, 5), "WEG3", False, 100, 2400.0),
    Trade(datetime(2024, 4, 4), "CEAB", True, 75, 155.0),
    Trade( datetime(2024, 4, 4), "WEG3", False, 150, 2450.0)
], datetime(2024, 4, 4)) == [('CEAB', 75), ('WEG3', -150)], 'The function is not returning the right values.'


Poderíamos ainda criar uma única função *total_by_attribute(trades, attribute, [optional args])*, que receberia o atributo chave pelo qual queremos agrupar o total e argumentos opcionais de forma que uma única função conseguiria retornar o total agrupado por todos os atributos: Data, Instrumento, Venda/Compra etc

## Problema 2

Implemente uma função que receba como parâmetro uma lista de preços de
fechamento de uma ação e retorne dois números: o número de vezes em que
a ação excedeu o valor máximo entre os fechamentos anteriores na lista, e o
número de vezes que ação ficou abaixo do mínimo dos fechamentos anteriores (o
primeiro valor da lista não conta em nenhuma das duas contagens).

**Solução:** Esse problema pode ser encarrado como um problema de *Sliding Window*. Basicamente utilizamos um ponteiro (ponteiro aqui é só uma variável que aponta para um elemento do array, não é o ponteiro de memória do C ou C++) que percorre a lista e marca a posição atual e outros dois ponteiros que vão marcar o maior e o menor valor. Abaixo está a implementação da solução e uma série de testes para validar.

Exemplo:

[20.50, 21.20, 22.40, 20.90, 21.57]

![Alt Text](slice_window.gif)


In [290]:
def get_maximums_and_minimums_quantities(prices_list: List[float]) -> Tuple[int]:

    if not prices_list: raise ValueError("Enter a valid price list! At least one value.")
    if len(prices_list) == 1: return(0,0)
    
    qty_of_highs:int = 0
    qty_of_low:int = 0
    pointer_min_value:int = 0
    pointer_max_value:int = 0

    for i in range(len(prices_list)-1):
        if (prices_list[i+1] - prices_list[pointer_max_value] > 0):
            qty_of_highs += 1
            pointer_max_value = i+1
        elif (prices_list[i+1] - prices_list[pointer_min_value] < 0):
            qty_of_low += 1
            pointer_min_value = i+1
        
    return(qty_of_highs, qty_of_low)


assert get_maximums_and_minimums_quantities([20.50, 21.20, 22.40, 20.90, 21.57]) == (2,0), 'The function is not returning the right values.'
assert get_maximums_and_minimums_quantities([20.52, 20.52, 19.81, 20.52]) == (0,1), 'The function is not returning the right values.'
assert get_maximums_and_minimums_quantities([30.00, 30.12, 29.55, 28.43, 27.93, 31.15, 31.16, 30.98, 32.55]) == (4,3), 'The function is not finding the list with the correct values.'
assert get_maximums_and_minimums_quantities([10.20, 10.50, 10.80, 10.90, 11.00, 10.70, 10.85]) == (4,0), 'The function is not finding the list with the correct values.'
assert get_maximums_and_minimums_quantities([25.10, 25.20, 25.15, 25.05, 25.30, 25.40, 25.50, 25.45]) == (4,1), 'The function is not finding the list with the correct values.'
assert get_maximums_and_minimums_quantities([18.30, 18.40, 18.25, 18.20, 18.10, 18.05, 18.10, 18.20, 18.30]) == (1,4), 'The function is not finding the list with the correct values.'
assert get_maximums_and_minimums_quantities([33.50, 33.20, 33.30, 33.45, 33.60, 33.40, 33.35, 33.25, 33.15]) == (1,2), 'The function is not finding the list with the correct values.'
assert get_maximums_and_minimums_quantities([14.90, 15.00, 14.85, 14.80, 14.75, 14.70, 14.95, 15.10, 15.05, 14.95]) == (2,4), 'The function is not finding the list with the correct values.'


## Problema 1

Implemente uma função que recebe uma lista L e um número inteiro (int) k e
faz k vezes uma “rotação” para a esquerda (se k for negativo,  é para a direita).

**Solução:** Esse problema é similar ao problema de rotacionar uma *linked list*. Podemos usar uma lógica semelhante. Vale a pena salientar que podemos usar a operação resto (%) para definir quantas rotação serão nescessarias. Exemplo, k = 7 em uma lista com 6 elementos ao invés de realizar 7 rotações podemos rotacionar 7%6=1 e o resultado será o mesmo. Esse resultado, 7%6=1, também serve para indicar qual *index* vamos mover para o início da lista. 

**Exemplo 1:**

lista: [1, 2, 3, 4, 5, 6]  
index: [0  1  2  3  4  5]  
k=7 -> 7%6=1 -> o índice 1 vai ser o novo *head*

resultado

lista rotacionada:        [2, 3, 4, 5, 6, 1]   
index antigo rotacionado: [1  2  3  4  5  0]  

![Alt Text](rotate_list.gif)

**Exemplo 2:**

lista: [1, 2, 3, 4, 5, 6]  
index: [0  1  2  3  4  5] 
Para k = -7 -> -7%6=5 -> o índice 5 é o novo *head*

lista rotacionada:        [6, 1, 2, 3, 4, 5]   
index antigo rotacionado: [5  0  1  2  3  4]  


Abaixo temos uma solução

In [407]:
def rotate(list: List[int], k:int) -> List[int]:
    if not list: raise ValueError("Enter a List")
    if not k: raise ValueError("Enter a k")

    k = k % len(list)

    if k == 0: return list

    rotated_list = [None] * len(list)

    if k < 0:
        for i in range(len(list)):
            rotated_index = (i + k) % len(list)
            rotated_list[rotated_index] = list[i]
    else:
        for i in range(len(list)):
            rotated_index = (i + k) % len(list)
            rotated_list[i] = list[rotated_index]

    return rotated_list


assert rotate([1,2,3,4,5,6], -7) == [6, 1, 2, 3, 4, 5], "The function doesn't work."
assert rotate([1,2,3,4,5,6], 7) == [2, 3, 4, 5, 6, 1], "The function doesn't work."
assert rotate([1,2,3,4,5], 2) == [3, 4, 5, 1, 2], "The function doesn't work."
assert rotate([1,2,3,4,5], -2) == [4, 5, 1, 2, 3], "The function doesn't work."

Felizmente o Python tem ferramentas próprias de *list slicing* que podem ajudar a resolver o problema mais facilmente. Abixo está outra solução utilizando o *list slicing*.

In [346]:
def rotate_with_slicing(list: List[int], k:int) -> List[int]:
    if not list: raise ValueError("Enter a List")
    if not k: raise ValueError("Enter a k")

    k = k % len(list)

    if k == 0: return list

    if k < 0:
        k = -k
        return list[-k:] + list[:-k]
    else:
        return list[k:] + list[:k]

assert rotate_with_slicing([1,2,3,4,5,6], -7) == [6, 1, 2, 3, 4, 5], "The function doesn't work."
assert rotate_with_slicing([1,2,3,4,5,6], 7) == [2, 3, 4, 5, 6, 1], "The function doesn't work."
assert rotate_with_slicing([1,2,3,4,5], 2) == [3, 4, 5, 1, 2], "The function doesn't work."
assert rotate_with_slicing([1,2,3,4,5], -2) == [4, 5, 1, 2, 3], "The function doesn't work."

## Problema 4

Implemente uma função que recebe uma lista de pares de números reais, que representam intervalos fechados, determine a união desses intervalos, e retorne uma nova lista de pares de números reais que represente o conjunto de intervalos fechados disjuntos (ou seja, com interseção vazia) que compõem essa união.

**Solução:** Esse problema pode ser resolvido utilizando um *Algoritmo Guloso*. A ideia é percorrer nossa lista de conjuntos e, dado um *critério guloso*, escolher intervalos da nossa lista. O critério guloso escolhido aqui é unir o intervalo *i* sempre que houver intersecção com o intervalo *i-1* e caso não tenha intersecção, adiciona a lista de intervalos disjuntos.

**Exemplo:**

Lista ordenada de Intervalos: [[-3.5, -1.5], [1, 3], [2, 5], [7, 10], [10, 15], [12, 20], [20, 22]]  

O último elemento do primeiro intervalo não é maior que o primeiro elemento do intervalo seguinte, ou seja, eles não se intersectam, então não vamos uní-los e sim adicionar os dois na nossa lista de intervalos disjuntos:

Lista output: [[-3.5, -1.5], [1,3]]

Agora vemos que o último item de [1,3] é maior que o primeiro elemento de [2,5], ou seja, esses intervalos têm uma intersecção, então podemos uní-los de forma que a lista de output fica:

Lista output: [[-3.5, -1.5], [1,5]]

Em seguida, vemos que o último elemento de [1,5] é menor que o primeiro elemento de [7,10] então eles não se intersectam, dessa forma nossa lista de output fica:

Lista output: [[-3.5, -1.5], [1,5], [7,10]]

Na sequência, vemos que o último elemento de [7,10] é igual ao primeiro elemento de [10,15] então eles se intersectam:

Lista output: [[-3.5, -1.5], [1,5], [7,15]]

Logo em seguida, vemos que o último elemento de [7,15] é maior que o primeiro elemento de [12,20] então os conjuntos se intersectam, dessa forma:

Lista output: [[-3.5, -1.5], [1,5], [7,20]]

Por fim, vemos que o último elemento de [7,20] é igual ao primeiro elemento de [20,22] então os dois intervelos tem intersecção e ficamos:

Lista output: [[-3.5, -1.5], [1,5], [7,22]]


In [374]:
def join_intervals(intervals: List[List[float]]) -> List[List[float]]:
    if not intervals: raise ValueError("Enter a Intervals List")

    intervals.sort()  

    output = [intervals[0]]

    for interval in intervals[1:]:
        if interval[0] <= output[-1][1]:
            output[-1][1] = max(output[-1][1], interval[1])
        else:
            output.append(interval)
            
    return output

assert join_intervals([[2, 5], [1, 3], [7, 10], [10, 15], [-3.5, -1.5], [20, 22], [12, 20]]) == [[-3.5, -1.5], [1, 5], [7, 22]], "The Functions doesn't works"
assert join_intervals([[1, 5], [2, 4], [6, 8], [7, 9], [10, 12]]) == [[1, 5], [6, 9], [10, 12]], "The Functions doesn't works"
assert join_intervals([[0, 3], [4, 6], [8, 10], [11, 15]]) == [[0, 3], [4, 6], [8, 10], [11, 15]], "The Functions doesn't works"
assert join_intervals([[-5, -1], [-3, 0], [2, 5], [7, 10]]) == [[-5, 0], [2, 5], [7, 10]], "The Functions doesn't works"


## Problema 3

Implemente uma função que recebe uma lista de *n* elementos contendo os números de 1 a *n* em uma ordem qualquer (em linguagem matemática, uma permutaçãoo de {1, . . . , n}), e retorne um set de listas disjuntas, representando os permutation cycles dessa permutação.

**Solução:** Esse problema foi mais desafiador, mas pesquisando vi que tem relação com álgebra abstrata. Encontrei um vídeo sobre o tema aqui [Cycle Notation of Permutations](https://www.youtube.com/watch?v=MpKG6FmcIHk&t=302s) e me ajudou bastante a pensar na solução. Unindo o vídeo e alguns algoritmos que encontrei na internet ([RosettaCode](https://rosettacode.org/wiki/Cycles_of_a_permutation#Python), [StackOverflow](https://stackoverflow.com/questions/36683413/permutations-producing-cycle-notation)) consegui uma solução satisfatória.

In [402]:
def permutation_cycles(permutation:List[int]) -> List[List[int]]:
    if not intervals: raise ValueError("Enter a Permutation!")
        
    visited = [False] * len(permutation)
    cycles = []

    for i in range(len(permutation)):
        if not visited[i]:
            cycle = []
            j = i
            
            while not visited[j]:
                cycle.append(j + 1)
                visited[j] = True
                j = permutation[j] - 1
                
            if cycle: cycles.append(cycle)
                
    return cycles

assert permutation_cycles([1, 2, 3, 4]) == [[1], [2], [3], [4]], "The Function doesn't works."
assert permutation_cycles([4, 3, 2, 1]) == [[1, 4], [2, 3]], "The Function doesn't works."
assert permutation_cycles([5, 1, 3, 4, 2]) == [[1, 5, 2], [3], [4]], "The Function doesn't works."
assert permutation_cycles([5, 3, 1, 2, 4]) == [[1, 5, 4, 2, 3]], "The Function doesn't works."